In [70]:
!pip install gtts
!pip install geocoder
!pip install pygame
!pip install pytesseract
#https://digi.bib.uni-mannheim.de/tesseract/
!pip install pywhatkit
!pip install twilio
!pip install opencv-python numpy
!pip install opencv-python matplotlib tensorflow tensorflow-hub Pillow
!pip install webcolors


In [71]:
import tkinter as tk
from gtts import gTTS
from datetime import datetime
import pygame
import geocoder
# Additional import for camera handling
from PIL import Image, ImageTk
import cv2
# import for sending email
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
import webbrowser
import pytesseract
import cv2
import numpy as np
from sklearn.cluster import KMeans
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
import tensorflow as tf
import matplotlib.pyplot as plt
from webcolors import rgb_to_name


In [72]:
pytesseract.get_tesseract_version()
import requests

In [73]:
def get_current_location():
    location = geocoder.ip('me')
    print(location)
get_current_location()

<[OK] Ipinfo - Geocode [Kitchener, Ontario, CA]>


In [96]:
# Initialize pygame
pygame.mixer.init()
import tensorflow_hub as hub

def get_weather(api_key, city):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": api_key,
    }
    response = requests.get(base_url, params=params)
    weather_data = response.json()
    return weather_data

def get_current_location():
    location = geocoder.ip('me')
    print(location)
    return location.city, location.latlng, location.country

def generate_emergency_message():
    # Get current location details
    city, province, country = get_current_location()
    
    # Get current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Generate emergency message
    emergency_message = f"aiCanSEE App: Emergency! Need help! Location: {city}, {province}, {country}. Timestamp: {timestamp}"
    
    return emergency_message

def play_alarm_sound():
    # Load the alarm sound file (replace 'alarm_sound.wav' with your desired sound file)
    pygame.mixer.music.load('./help/alarm.mp3')

    # Play the alarm sound
    pygame.mixer.music.play()
def text_to_speech(text, output_path):
    tts = gTTS(text=text, lang='en')
    tts.save(output_path)

def suggest_outfit(api_key):
    city,_,_ = get_current_location()
    weather_data = get_weather(api_key, city)
    
    # Check if the API call was successful
    if weather_data.get("cod") != 200:
        return f"Unable to retrieve weather information. Error code: {weather_data.get('cod')}"
    print(weather_data)
    # Extract relevant weather information (e.g., temperature, feels_like)
    temperature_kelvin = weather_data["main"]["temp"]
    feels_like_kelvin = weather_data["main"]["feels_like"]

    temperature_celsius = round(temperature_kelvin - 273.15)
    feels_like_celsius = round(feels_like_kelvin - 273.15)

    # Determine suitable outfit based on Canada's weather conditions
    if feels_like_celsius > 25:
        outfit_suggestion = "You may want to wear light and breathable clothing."
    elif 20 <= feels_like_celsius <= 25:
        outfit_suggestion = "Moderate clothing would be suitable."
    elif 15 <= feels_like_celsius < 20:
        outfit_suggestion = "A light jacket or sweater may be comfortable."
    elif 10 <= feels_like_celsius < 15:
        outfit_suggestion = "Consider wearing a sweater or light coat."
    elif 5 <= feels_like_celsius < 10:
        outfit_suggestion = "A warm coat and layers would be advisable."
    elif 0 <= feels_like_celsius < 5:
        outfit_suggestion = "Dress warmly with a winter coat and layers."
    else:
        outfit_suggestion = "Extreme cold! Bundle up with heavy winter clothing."

    # Format temperature information
    temperature_str = f"{temperature_celsius} degrees Celsius"
    feels_like_str = f"{feels_like_celsius} degrees Celsius"

    # Convert the result to audio
    text_to_speech = f"You are currently in {city}. The current temperature is {temperature_str}, but it feels like {feels_like_str}. {outfit_suggestion}. Have a great day!"
    tts = gTTS(text=text_to_speech, lang='en')

    # Convert the result to audio
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    # Save the audio file
    audio_path = f"output_{timestamp}.mp3"
    tts.save(audio_path)
    return text_to_speech,audio_path

def playaudio(audio_path):
    # Play the audio
    pygame.mixer.music.load(audio_path)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        continue
    
def execute_suggestion():
    api_key = "add_api_key_here"
    result,audio_path = suggest_outfit(api_key)
    print("Weather"+ result)
    playaudio(audio_path)

def capture_image(x):
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    image_path = "captured_image.jpg"
    cv2.imwrite(image_path, frame)
    cap.release()

    # Send the captured image via email
    if x==1:
        return image_path
    else:
        send_email(image_path)
        # Perform OCR on the captured image
        text_content = perform_ocr(image_path)
        # Display the captured image and OCR text
        display_captured_image(image_path, text_content)        

def send_email(image_path):
    # Set up the email server
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()

    # Log in to your email account
    server.login(sender_email, sender_password)

    # Create the email message
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = 'Captured Image from aiCanSEE App'

    # Attach the image to the email
    with open(image_path, 'rb') as file:
        image_data = file.read()
        image_attachment = MIMEImage(image_data, name='captured_image.jpg')
        message.attach(image_attachment)

    # Send the email
    server.sendmail(sender_email, receiver_email, message.as_string())

    # Close the server connection
    server.quit()

def perform_ocr(image_path):
    # Use pytesseract to perform OCR on the image
    text_content = pytesseract.image_to_string(Image.open(image_path))
    return text_content


def display_captured_image(image_path, text_content):
    # Display the captured image in a new window
    captured_image = Image.open(image_path)
    captured_image = captured_image.resize((300, 300), Image.LANCZOS)
    photo = ImageTk.PhotoImage(captured_image)

    image_window = tk.Toplevel(root)
    image_label = tk.Label(image_window, image=photo)
    image_label.photo = photo
    image_label.pack()
    if(text_content):
        # Convert the description to audio
        text_to_speech = f"{text_content}"
        tts = gTTS(text=text_to_speech, lang='en')

        # Convert the result to audio
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        # Save the audio file
        audio_path = f"ocr_output_{timestamp}.mp3"
        tts.save(audio_path)

        # Play the audio
        pygame.mixer.music.load(audio_path)
        pygame.mixer.music.play()
        # Display the extracted text content
        text_label = tk.Label(image_window, text=text_content, wraplength=300)
        text_label.pack()

def generate_emergency_message():
    # Get current location details
    location = get_current_location()
    
    # Get current timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Generate emergency message
    emergency_message = f"aiCanSEE App: Emergency! Need help! Location is: {location}. Time of message: {timestamp}"
   
    return emergency_message

def send_whatsapp_message(phone_number, message):
    # Format the phone number (remove non-numeric characters)
    formatted_phone_number = ''.join(c for c in phone_number if c.isdigit())

    # Create the WhatsApp message link
    whatsapp_link = f"https://wa.me/{formatted_phone_number}?text={message}"

    # Open the link or display it for the user to click
    webbrowser.open(whatsapp_link)

def send_emergency_message():
    emergency_message = generate_emergency_message()
    # Display the emergency message in the app
    phone_number = "type a number here"  # emergency contact number
    send_whatsapp_message(phone_number, emergency_message)

def find_dominant_color(image_path, k=1):
    # Load image
    image = cv2.imread(image_path)
    # Convert image pixels to a flat array of BGR values
    pixels = image.reshape((-1, 3))
    
    # Convert to float for more accurate k-means prediction
    pixels = np.float32(pixels)
    
    # Use k-means clustering to find most dominant color(s)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Convert centers to integers (BGR values)
    centers = np.uint8(centers)
    
    # Get the most dominant color
    dominant_color = centers[0].tolist()
    
    return dominant_color


def describe_color(rgb_values):

    base_url = "https://www.thecolorapi.com/id"
    # Construct the URL with RGB values
    url = f"{base_url}?rgb={','.join(map(str, rgb_values))}&format=json"

    # Make the GET request
    response = requests.get(url)
    color_data = response.json()
    color_name = color_data["name"]["value"]
    return f"The color is {color_name}."

def capture_and_analyze_color():
    
    image_path = capture_image(1)
    
    # Find dominant color
    dominant_color = find_dominant_color(image_path, k=1)
    
    # Convert BGR to RGB
    dominant_color_rgb = dominant_color[::-1]
    
    print(f"Dominant Color (RGB): {dominant_color_rgb}")
    color_description = describe_color(dominant_color_rgb)
    print(color_description)
    # Convert the result to audio
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    # Convert the color description to audio
    output_path = f"color_description_{timestamp}.mp3"
    text_to_speech(color_description, output_path)
    playaudio(output_path)

def load_image(image_path):
    img = Image.open(image_path).resize((224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array / 255.0 
def predict_description(image_path):
    model_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
    model = hub.load(model_url)
    image = load_image(image_path)
    predictions = model(image)
    class_id = tf.argmax(predictions[0]).numpy()
    labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    with open(labels_path) as f:
        labels = f.read().splitlines()
    object = labels[class_id]
    return object

def evaluate():
    # Capture image (assuming you have a capture_image function)
    image_path = capture_image(1)

    # Get textual description from the model
    description = predict_description(image_path)
    print(description)

    # Convert the description to audio
    text_to_speech = f"It might be  {description}"
    tts = gTTS(text=text_to_speech, lang='en')

    # Convert the result to audio
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    # Save the audio file
    audio_path = f"output_{timestamp}.mp3"
    tts.save(audio_path)

    # Play the audio
    pygame.mixer.music.load(audio_path)
    pygame.mixer.music.play()

# Your email credentials
sender_email = "sender@gmail.com"
sender_password = "password"
receiver_email = "receiver@gmail.com"

# Global counters for taps
capture_tap_count = 0
alarm_tap_count = 0

# Capture Image Tap Handler
def handle_capture_tap():
    global capture_tap_count
    capture_tap_count += 1
    if capture_tap_count == 2:
        # Execute capture image function
        capture_image(2)
        capture_tap_count = 0  # Reset tap count

# Alarm Tap Handler
def handle_alarm_tap():
    global alarm_tap_count
    alarm_tap_count += 1
    if alarm_tap_count == 3:
        # Execute play alarm sound function
        play_alarm_sound()
        alarm_tap_count = 0  # Reset tap count
# Tkinter setup
root = tk.Tk()
root.title("aICanSEE")

# Button to execute the weather suggestion
execute_button = tk.Button(root, text="1. Weather", command=execute_suggestion, height=5, width=20)
execute_button.grid(row=0, column=0, padx=10, pady=10)

# Button for capturing an image, configured for double tap
capture_button = tk.Button(root, text="2. Capture and Send(Tap 2)", command=handle_capture_tap, height=5, width=20)
capture_button.grid(row=0, column=1, padx=10, pady=10)

analyze_color_button = tk.Button(root, text="3. Find Color", command=capture_and_analyze_color, height=5, width=20)
analyze_color_button.grid(row=1, column=0, padx=10, pady=10)

# Button to send emergency message
help_button = tk.Button(root, text="4. Help Me", command=send_emergency_message, height=5, width=20)
help_button.grid(row=1, column=1, columnspan=2, padx=10, pady=10)

# Button to execute the evaluation
eval_button = tk.Button(root,text="5. Tell me the object?",command=evaluate, height=5, width=20)
eval_button.grid(row=2, column=0, padx=10, pady=10)

# Button for triggering an alarm, configured for triple tap
alarm_button = tk.Button(root, text="6. Emergency Alarm (Tap 3)", command=handle_alarm_tap, height=5, width=20)
alarm_button.grid(row=2, column=1, padx=10, pady=10)


root.mainloop()